In [9]:
from matplotlib import pyplot as plt
from fedhex.io import RootLoader

In [3]:
root_path = "./root/10x10box_10k_jun2023/"
loader = RootLoader(root_path)

./root/10x10box_10k_jun2023//Phi_100_omega_0p25/2023-06-12-11-53-45/v1p0-78-8cf4/ATTOAODv1p2c2_0.root
./root/10x10box_10k_jun2023//Phi_1280_omega_9p025/2023-06-12-11-53-59/v1p0-78-8cf4/ATTOAODv1p2c2_0.root
./root/10x10box_10k_jun2023//Phi_3050_omega_6p1/2023-06-12-11-54-18/v1p0-78-8cf4/ATTOAODv1p2c2_0.root
./root/10x10box_10k_jun2023//Phi_5410_omega_9p025/2023-06-12-11-54-50/v1p0-78-8cf4/ATTOAODv1p2c2_0.root
./root/10x10box_10k_jun2023//Phi_4230_omega_1p225/2023-06-12-11-54-29/v1p0-78-8cf4/ATTOAODv1p2c2_0.root
./root/10x10box_10k_jun2023//Phi_1280_omega_3p175/2023-06-12-11-53-55/v1p0-78-8cf4/ATTOAODv1p2c2_0.root
./root/10x10box_10k_jun2023//Phi_4230_omega_2p2/2023-06-12-11-54-30/v1p0-78-8cf4/ATTOAODv1p2c2_0.root
./root/10x10box_10k_jun2023//Phi_100_omega_6p1/2023-06-12-11-53-49/v1p0-78-8cf4/ATTOAODv1p2c2_0.root
./root/10x10box_10k_jun2023//Phi_1870_omega_9p025/2023-06-12-11-54-06/v1p0-78-8cf4/ATTOAODv1p2c2_0.root
--- ^ empty file ^ ---
./root/10x10box_10k_jun2023//Phi_4230_omega_6p1/20

In [8]:
samples, labels = loader.recover_preproc()